<a href="https://colab.research.google.com/github/Shakilkhan24/Playground_DL/blob/main/custom_tool_with_rag_(agent).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
api_key=''

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install chromadb

In [3]:
from pydantic.v1 import BaseModel, Field
from langchain_community.document_loaders import NotebookLoader
loader = NotebookLoader(
    "/content/kaggle-llama-3-8b-unsloth-notebook.ipynb",
    include_outputs=True,
    max_output_length=20,
    remove_newline=True,
)

In [31]:
from pydantic.v1 import validator

In [ ]:
print(loader.load()[0].page_content)

In [ ]:
import osqp
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain import PromptTemplate

chat = ChatOpenAI(temperature=0, openai_api_key=api_key)
embedding = OpenAIEmbeddings(openai_api_key=api_key)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
 )
docs=loader.load()
text_chunks = splitter.split_documents(docs)

db = Chroma.from_documents(text_chunks, embedding,persist_directory="./chroma_db")

In [ ]:
from google.colab import files
local_file_path = "/content/chroma_db"
files.download(local_file_path)

In [ ]:
db=db3 = Chroma(persist_directory="/content/chroma_db", embedding_function=embedding)
result=db.similarity_search('adding lora adapters')
for i in result:
  print(i.page_content)


In [54]:
class RAG:
  def __init__(self,chroma_path,prompt):
    self.prompt=prompt
    self.db= Chroma(persist_directory=chroma_path, embedding_function=embedding)
  def run(self):
    template=""" Based on the retrieved datas : <context> {retrieved} </context>, answer the following question :<context> {question}
    </context> and generate code snippet for better understanding if needed or asked.
    """
    similar_results=self.db.similarity_search(self.prompt)
    retrieved_data=' '
    for i in similar_results:
      retrieved_data+=i.page_content

    chat = ChatOpenAI(temperature=0, openai_api_key=api_key)
    prompt_template=PromptTemplate.from_template(template)
    final_prompt=prompt_template.format(retrieved=retrieved_data,question=self.prompt)
    answer=chat.invoke(final_prompt)
    return answer


In [ ]:
r=RAG(chroma_path="/content/chroma_db",prompt='how to add eos token in the case of terminating generation?')
answer=r.run()
print(answer.content)

In [55]:
from langchain.tools import BaseTool, StructuredTool, tool


def func(prompt:str):   # another argument is chroma_path:str
  r=RAG(chroma_path='/content/chroma_db',prompt=prompt)
  result=r.run()
  return result

class Input(BaseModel):
    # chroma_path: str = Field(..., description='Provide a valid Chroma vector database path')
    prompt: str = Field(..., description='Provide a string prompt text')

    # @validator('chroma_path')
    # def validate_chroma_path(cls, chroma_path):
    #     if not chroma_path or not chroma_path.endswith('/chroma_db'):
    #         raise ValueError('Vector database path must end with: /chroma_db ')
    #     return chroma_path

retriever_tool = StructuredTool.from_function(
    func=func,
    name="chroma_data_base to answer generation",
    description="retrive data from chroma vector database and then based on the retrieved data generating answer through llm invokation",
    args_schema=Input,
    return_direct=True,
)



In [ ]:
print(retriever_tool.name)
print(retriever_tool.description)
print(retriever_tool.args)

In [ ]:
!pip install langchainhub
!pip install langchain-experimental

In [56]:
from langchain import hub
from langchain.agents import Tool, AgentExecutor, create_react_agent
prompt = hub.pull('hwchase17/react')

tools = [retriever_tool]

agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2
)




In [ ]:
agent_executor.invoke({'input':'how to load fourbits model? '})